# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.tail()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
556,556,alaghsas,17.0187,8.0168,37.94,11,0,5.14,NE,1728578790
557,557,salinas,36.6777,-121.6555,19.35,88,100,2.57,US,1728578791
558,558,kirakira,-10.4544,161.9205,25.60,88,96,3.37,SB,1728578792
559,559,mahanoro,-19.9000,48.8000,21.10,80,12,1.12,MG,1728578793
560,560,selfoss,63.9331,-20.9971,2.80,54,87,6.33,IS,1728578798


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height =500,
    size = 'Humidity',
    scale = 1.0,
    color = 'City',
    alpha = 0.5
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_less_than = 30
temp_greater_than = 23

ideal_temps = city_data_df[(city_data_df['Max Temp'] < temp_less_than) & (city_data_df['Max Temp'] > temp_greater_than)]

# Drop any rows with null values
ideal_temps = ideal_temps.dropna()

# Display sample data
ideal_temps

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,libreville,0.3925,9.4537,29.01,79,20,5.14,GA,1728577943
17,17,tanda,26.5500,82.6500,26.06,59,97,2.29,IN,1728577962
24,24,antsiranana,-12.2787,49.2917,24.43,71,10,9.00,MG,1728577970
28,28,wanning,18.8003,110.3967,24.10,92,42,2.53,CN,1728577976
31,31,ribeira grande,38.5167,-28.7000,23.76,73,75,12.35,PT,1728577979
...,...,...,...,...,...,...,...,...,...,...
544,544,prado,-17.3411,-39.2208,27.63,65,95,4.64,BR,1728578775
546,546,bhimunipatnam,17.8833,83.4333,28.73,81,70,1.84,IN,1728578777
549,549,watampone,-4.5386,120.3279,25.12,93,100,1.28,ID,1728578781
553,553,shakawe,-18.3667,21.8500,28.53,13,5,3.20,BW,1728578787


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity# YOUR CODE HERE
hotel_df = ideal_temps[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,libreville,GA,0.3925,9.4537,79,
17,tanda,IN,26.5500,82.6500,59,
24,antsiranana,MG,-12.2787,49.2917,71,
28,wanning,CN,18.8003,110.3967,92,
31,ribeira grande,PT,38.5167,-28.7000,73,
...,...,...,...,...,...,...
544,prado,BR,-17.3411,-39.2208,65,
546,bhimunipatnam,IN,17.8833,83.4333,81,
549,watampone,ID,-4.5386,120.3279,93,
553,shakawe,BW,-18.3667,21.8500,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1
apiKey = geoapify_key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Update the params dictionary with the current city's latitude, longitude, and API key
    params = {
        'categories': categories,
        'limit': limit,
        'filter': f"circle:{lon},{lat},{radius}",
        'bias': f"proximity:{lon},{lat}",
        'apiKey': geoapify_key
    }

    # Make the API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results for each city
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
libreville - nearest hotel: No hotel found
tanda - nearest hotel: No hotel found
antsiranana - nearest hotel: Central
wanning - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
utrik - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
samalut - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
miri - nearest hotel: One Times One Inn
lubango - nearest hotel: Novo Hotel
copala - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: Hotel do Sol
lihue - nearest hotel: Kauai Palms
gurmatkal - nearest hotel: Samrat Hotel
enewetak - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
kavaratti - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
georgetown - nearest hotel: Page 63 hostel
kawalu - nearest hotel: Hotel Surya
loango - nearest hotel: Hotel des Aigles
mundargi - n

,City,Country,Lat,Lng,Humidity,Hotel Name
0,libreville,GA,0.3925,9.4537,79,No hotel found
17,tanda,IN,26.5500,82.6500,59,No hotel found
24,antsiranana,MG,-12.2787,49.2917,71,Central
28,wanning,CN,18.8003,110.3967,92,No hotel found
31,ribeira grande,PT,38.5167,-28.7000,73,Hospedaria JSF
...,...,...,...,...,...,...
544,prado,BR,-17.3411,-39.2208,65,Pousada Recanto do Prado
546,bhimunipatnam,IN,17.8833,83.4333,81,Novotel
549,watampone,ID,-4.5386,120.3279,93,Hotel Rio Rita
553,shakawe,BW,-18.3667,21.8500,13,EdenWood guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height =500,
    scale = 1.0,
    color = 'Hotel Name',
    alpha = 0.5,
    hover_cols = ['City','Country']
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)